## Linear Elasticity for Cantilever Beam

In [ ]:
using Gridap, Gridap.CellData, SparseArrays
using NearestNeighbors, Printf  # make sure this is at the top of your file
using Makie,GLMakie, LaTeXStrings, Makie.GeometryBasics

In [ ]:
include("plot_design.jl")
include("OC.jl")
include("SIMP_Inter.jl")

## 1. Problem
### Let's consider the 3D cantilever beam as follows:
<img src="imagines/3D_Cantilever.png" style="width:400px;height:250px"/>

The dimensions of beam are $L$ and $W$. 

This beam is fixed at $x=0$ and is applied a uniformly distributed load at $x=L$. The magnitude of the distributed load is $\mathbf{F} = [0, -1, 0]^T$. 

## Strong Form

**Equilibrium equation:**
$$
\nabla \cdot \boldsymbol{\sigma} + \mathbf{b} = \mathbf{0} \text{ in } \Omega,
$$

**Kinematics equation (strain–displacement relation):**
$$
\boldsymbol{\varepsilon} = \nabla_S \mathbf{u} = \frac{1}{2} \Big[ \nabla \mathbf{u} + (\nabla \mathbf{u})^T \Big].
$$

**Constitutive equation (stress–strain relation):**
$$
\boldsymbol{\sigma} = \mathbf{C} \boldsymbol{\varepsilon} 
\quad \text{or} \quad 
\boldsymbol{\sigma} = \lambda \, \text{tr}(\boldsymbol{\varepsilon})\, \mathbf{I} + 2\mu\, \boldsymbol{\varepsilon}.
$$

At the face with $x=L$ which is a boundary condition denoted by $\Gamma$, we have:

$$
\boldsymbol{\sigma} \cdot \mathbf{n} = \mathbf{F} = \begin{bmatrix}
0 & -1 & 0
\end{bmatrix}^T
 \text{ on } \Gamma,
$$

where $\boldsymbol{\sigma}$ is the stress tensor, $\mathbf{b}$ is the body force, $\mathbf{F}$ is the applied traction, and $\mathbf{n}$ is the outward unit normal.


## Derivation of the Weak Form from the Strong Form

### Step 1: Multiply by a Test Function
To derive the weak form, multiply the strong form by a test function $\mathbf{v} \in H^1(\Omega)$ (with $\mathbf{v} = \mathbf{0}$ on Dirichlet boundaries) and integrate over the domain $\Omega$:

$$
\int_{\Omega} (\nabla \cdot \boldsymbol{\sigma}) \cdot \mathbf{v} \, d\Omega = -\int_{\Omega} \mathbf{b} \cdot \mathbf{v} \, d\Omega.
$$
In this example, we assume that $\mathbf{b} = \mathbf{0}$, the we have:

$$
\int_{\Omega} (\nabla \cdot \boldsymbol{\sigma}) \cdot \mathbf{v} \, d\Omega = 0.
$$


### Step 2: Apply the Divergence Theorem
We have the equivalent form of $(\nabla \cdot \boldsymbol{\sigma}) \cdot \mathbf{v} $ as follows:

$$
(\nabla \cdot \boldsymbol{\sigma}) \cdot \mathbf{v} = \nabla \cdot (\boldsymbol{\sigma} \cdot \mathbf{v}) - \boldsymbol{\sigma} : \nabla \mathbf{v}.
$$
Then we have,
$$
\int_{\Omega} (\nabla_S^T \boldsymbol{\sigma}) \cdot \mathbf{v} \, d\Omega = \int_{\Omega} \left[ \nabla \cdot (\boldsymbol{\sigma} \cdot \mathbf{v}) - \boldsymbol{\sigma} : \nabla \mathbf{v} \right] \, d\Omega.
$$

Apply the divergence theorem to $ \int_{\Omega} \nabla \cdot (\boldsymbol{\sigma} \cdot \mathbf{v}) \, d\Omega$:

$$
\int_{\Omega} \nabla \cdot (\boldsymbol{\sigma} \cdot \mathbf{v}) \, d\Omega = \int_{\partial \Omega} (\boldsymbol{\sigma} \cdot \mathbf{v}) \cdot \mathbf{n} \, d\Gamma = \int_{\partial \Omega} (\boldsymbol{\sigma} \cdot \mathbf{n}) \cdot \mathbf{v} \, d\Gamma.
$$

Because, we set the boundary condition $\boldsymbol{\sigma} \cdot \mathbf{n} = \mathbf{F}$ on $\Gamma$, then we have:

$$
\int_{\partial \Omega} (\boldsymbol{\sigma} \cdot \mathbf{n}) \cdot \mathbf{v} \, d\Gamma = \int_{\Gamma} \mathbf{F} \cdot \mathbf{v} \, d\Gamma.
$$

Thus:

$$
\int_{\Omega} (\nabla_S^T \boldsymbol{\sigma}) \cdot \mathbf{v} \, d\Omega = \int_{\Gamma} \mathbf{F} \cdot \mathbf{v} \, d\Gamma - \int_{\Omega} \boldsymbol{\sigma} : \nabla \mathbf{v} \, d\Omega = 0.
$$



### Step 3: Relate Stress to Strain
The stress $\boldsymbol{\sigma}$ depends on the strain $\boldsymbol{\varepsilon}(\mathbf{u})$, where the strain tensor is:

$$
\boldsymbol{\varepsilon}(\mathbf{u}) = \frac{1}{2} \left( \nabla \mathbf{u} + (\nabla \mathbf{u})^T \right).
$$

Similarly, for the test function:

$$
\boldsymbol{\varepsilon}(\mathbf{v}) = \frac{1}{2} \left( \nabla \mathbf{v} + (\nabla \mathbf{v})^T \right).
$$

$$
\nabla \mathbf{v} = \boldsymbol{\varepsilon}(\mathbf{v}) + \frac{1}{2} \left( \nabla \mathbf{v} - (\nabla \mathbf{v})^T \right).
$$

$$\nabla \mathbf{v} = \boldsymbol{\varepsilon}(\mathbf{v}) + \boldsymbol{\omega}(\mathbf{v})$$

where $\boldsymbol{\omega}(\mathbf{v}) = \frac{1}{2} \left( \nabla \mathbf{v} - (\nabla \mathbf{v})^T \right)$, then we have:

$$
\boldsymbol{\sigma} : \nabla \mathbf{v} = \boldsymbol{\sigma} : \boldsymbol{\varepsilon}(\mathbf{v}) + \boldsymbol{\sigma} : \boldsymbol{\omega}(\mathbf{v}),
$$
because $\boldsymbol{\omega}(\mathbf{v}) $ is skew-symmetric and $\boldsymbol{\sigma}$ is symmetric, then $\boldsymbol{\sigma} : \boldsymbol{\omega}(\mathbf{v}) = 0$. Thus:

$$
\int_{\Omega} \boldsymbol{\sigma} : \nabla \mathbf{v} \, d\Omega = \int_{\Omega} \boldsymbol{\sigma} : \boldsymbol{\varepsilon}(\mathbf{v}) \, d\Omega.
$$



### Step 4: Formulate the Weak Form
Now, we have the weak form as follows:
$$
\int_{\Omega} \boldsymbol{\sigma}(\boldsymbol{\varepsilon}(\mathbf{u})) : \boldsymbol{\varepsilon}(\mathbf{v}) \, d\Omega = \int_{\Gamma} \mathbf{F} \cdot \mathbf{v} \, d\Gamma.
$$


### Final Weak Form to be used in Gridap.jl
Then, before using Gridap.jl, we define the bilinear form. Find $\mathbf{u}$ such that for all test functions $\mathbf{v}$:

$$
a(\mathbf{u}, \mathbf{v}) = l(\mathbf{v}),
$$

where:

$$
a(\mathbf{u}, \mathbf{v}) = \int_{\Omega} \big[ \boldsymbol{\sigma}(\boldsymbol{\varepsilon}(\mathbf{u})) : \boldsymbol{\varepsilon}(\mathbf{v}) \big] \, d\Omega,
\quad
l(\mathbf{v}) = \int_{\Gamma} \mathbf{F} \cdot \mathbf{v} \, d\Gamma.
$$

## Inputs

In [ ]:
# ==============================================
L = 60 # Length
W = 20 # Height
domain = (0, L, 0, W) # sizes along X, Y
partition = (L*2, W*2) # mesh sizes
model = CartesianDiscreteModel(domain, partition)
# writevtk(model, "Cantilever_Beam_2D")
# Tag the left face (x = 0); right face (x = L)
labels = get_face_labeling(model)
add_tag_from_tags!(labels, "left", [1, 3, 7]) # 
add_tag_from_tags!(labels, "right", [2, 4, 8]) # 
# Integrating on the domain Ω
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω, degree)
# neumann boundaries
Γ = BoundaryTriangulation(model, tags="right")
dΓ = Measure(Γ, degree)
# Vector-valued FE space
order = 1
reffe = ReferenceFE(lagrangian, VectorValue{2,Float64}, order)
Vₕ = TestFESpace(Ω, reffe; conformity=:H1, dirichlet_tags="left")
Uₕ = TrialFESpace(Vₕ)
# Constitutive law for LINEAR ELASTICITY 
const E = 119e3 # Young's modulus
const ν = 0.3   # Poisson's ratio
const λ = (E * ν) / ((1 + ν) * (1 - 2 * ν)) # Lamé parameters
const μ = E / (2 * (1 + ν)) # Lamé parameters
ε₀(u) = 0.5 * (∇(u) + transpose(∇(u))) # Strain; In Gridap this can be automatically defined in Gridap.ε
σ(ε₀) = λ * tr(ε₀) * one(ε₀) + 2μ * ε₀ # Stress


## Filters

In [ ]:

num_ele = num_cells(Ω) # Number of elements
volfrac = 0.4
p = 3
max_iter = 50
# Compute centroids of elements and store in a matrix
cell_coords = get_cell_coordinates(Ω) # Get coordinates of vertices for each cell
no_ele = length(cell_coords) # Number of elements
centroid_matrix = zeros(2, no_ele) # Matrix to store centroids (no_ele × 2)

for (cell_id, coords) in enumerate(cell_coords)
    # Compute centroid by averaging vertex coordinates
    centroid = sum(coords) / length(coords)
    centroid_matrix[:, cell_id] = [centroid[1], centroid[2]] # Store x, y coordinates
end

# Number of neighbors for each element
filter_radius = 2
tol = 1e-3
search_dist = filter_radius - tol
balltree = BallTree(centroid_matrix)
OPT_H = zeros(num_ele, num_ele)
for iel in 1:num_ele
    near_ele = inrange(balltree, centroid_matrix[:, iel], search_dist)
    dist = sqrt.(sum((centroid_matrix[:, near_ele] .- centroid_matrix[:, iel]) .^ 2, dims=1))
    num = 1 .- dist ./ filter_radius
    den = sum(num)
    OPT_H[iel, near_ele] = num / den
end
H = sparse(OPT_H);

## Run topology optimization

In [ ]:

for iter = 1:max_iter
    if iter == 1
        ρ_new = 0.5 * ones(num_ele)
    end
    ρ_old = copy(ρ_new)
    ρ_new_fil = H * ρ_new # Filter
    # SIMP
    ρ_SIMP, dρ_dp = SIMP_Inter(ρ_new_fil, p, Ω)
    # ===FE-ANALYSIS
    F(x) = VectorValue(0.0, -1) # I want to apply a unit load along Y axis to each node on the right face
    l(v) = ∫(F ⋅ v) * dΓ # Right-hand size
    a(u, v) = ∫(ρ_SIMP * (σ ∘ ε₀(u)) ⊙ ε₀(v)) * dΩ # Left-hand size; (∘) Composite functions

    # Solution of the FE problem
    op = AffineFEOperator(a, l, Uₕ, Vₕ)
    uh = solve(op)
    C = sum(∫(ρ_SIMP * (σ ∘ ε₀(uh)) ⊙ ε₀(uh)) * dΩ) # Compliance
    dC_fil = get_contribution(∫(-dρ_dp * (σ ∘ ε₀(uh)) ⊙ ε₀(uh)) * dΩ, Ω) # Sensitivity of compliance w.r.t rho
    dC = transpose(H) * (dC_fil[:])

    ρ_new = OC(ρ_new, volfrac, dC, num_ele)
    change_obj = maximum(abs.(ρ_new .- ρ_old))
    figs = plot_design(ρ_new, cell_coords, iter, L, W)

    display(figs)

    println("It. ", iter, ", Obj = ", @sprintf("%.5e", C), ", Vol.: ", @sprintf("%.5e", sum(ρ_new) / num_ele), ", ch.: ", @sprintf("%.5e", change_obj))
    if iter == max_iter
        writevtk(Ω, "Cantilever_Beam_2D_TOP", cellfields=["rho" => ρ_SIMP])
    end
end